In [ ]:
from __future__ import print_function

#import tensorflow as tf
import numpy as np

## Parameters

In [ ]:
PLOTTING_SUPPORT = True
SET_EULER_PARAMETERS = False

# Handle arguments (When executed as .py script)
import sys
argv = sys.argv[:]
if len(argv) > 1:
  script_path = argv.pop(0)
  if "--euler" in argv:
    SET_EULER_PARAMETERS = True
    PLOTTING_SUPPORT = False
    print("Parameters set for execution on euler cluster")
    argv.remove("--euler")

In [ ]:
DATA_FOLDER = "/home/daniel/Downloads/Raw-Waves/"
DATA_FILENAME="001_Session1_FilterTrigCh_RawCh.mat"
DATA2_FILENAME="001_Session2_FilterTrigCh_RawCh.mat"
DATA3_FILENAME="034_Session1_FilterTrigCh_RawCh.mat"
BATCH_SIZE = 20
BATCH_LIMIT_PER_STEP = 10
TRAINING_DATA_LENGTH = "max"
VAL_DATA_LENGTH = 1200000
TEST_DATA_LENGTH = 2000
SAMPLING = 1

MAX_STEPS = 1000

VAL_EVERY_N_STEPS = 1
VAL_STEP_TOLERANCE = 100

class ModelParams:
  def __init__(self):
    self.DROPOUT = 1.0
  def __str__(self):
    return str(self.__dict__)
  def __eq__(self, other): 
    return self.__dict__ == other.__dict__
  def __ne__(self, other):
    return not self.__eq__(other)
MP = ModelParams()

SAVE_DIR = "/home/daniel/Desktop/feedforward-model/"
SAVE_FILE = "model.ckpt"
TENSORBOARD_DIR = "/home/daniel/tensorboard"

In [ ]:
if SET_EULER_PARAMETERS:
    DATA_FOLDER = "/cluster/home/dugasd/Data_200Hz/"
    SAVE_DIR = "/cluster/home/dugasd/tf-lstm-model/"
    TENSORBOARD_DIR = None
    
    BATCH_SIZE = 20
    BATCH_LIMIT_PER_STEP = 10000
    MAX_STEPS = 1000000
    VAL_STEP_TOLERANCE = 100

In [ ]:
if PLOTTING_SUPPORT:
  import matplotlib.pyplot as plt
  from cycler import cycler
  if SAMPLING > 0:
      plotting_function = plt.step
  else:
      plotting_function = plt.plot

In [ ]:
SAVE_PATH = SAVE_DIR+SAVE_FILE

## Datasets

In [ ]:
if True:
  raw_wave = []
  raw_wave2 = []
  raw_wave3 = []

  import scipy.io
  mat = scipy.io.loadmat(DATA_FOLDER+DATA_FILENAME)
  raw_wave = mat.get('data')[0]
  raw_wave = raw_wave[::SAMPLING]
  wave_indices = mat.get('wave')[0]

  if DATA2_FILENAME is not None:
      mat = scipy.io.loadmat(DATA_FOLDER+DATA2_FILENAME)
      raw_wave2 = mat.get('data')[0]
      raw_wave2 = raw_wave2[::SAMPLING]
  if DATA3_FILENAME is not None:
      mat = scipy.io.loadmat(DATA_FOLDER+DATA3_FILENAME)
      raw_wave3 = mat.get('data')[0]
      raw_wave3 = raw_wave3[::SAMPLING]
    
  # Save some memory
  del mat

In [ ]:
if PLOTTING_SUPPORT:
  plt.figure()
  plotting_function(range(len(raw_wave)),raw_wave,label="raw_wave")
  plt.ylim([-1000,1000])
  plt.show()